In [1]:
%pwd
%cd /home/arno/Projects/EEGDecodingTest/My

/home/arno/Projects/EEGDecodingTest/My


/home/arno/conda/envs/MNE-EEG/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
#region 已经训练过一个模型从EEG latent预测embedding，读取所有的（character，训练embed）
import os
import pickle


root_dir = 'Data/qwen-characterSplit'
sub=[8] # ! Mask

characters=[]
for sub_dir in os.listdir(root_dir):
    if all(item not in sub_dir for item in map(str, sub)):
        continue
    else:
    # if "08" in sub_dir:
    # if "04" in sub_dir or "05" in sub_dir or "08" in sub_dir or True:
        sub_dir_path = os.path.join(root_dir, sub_dir)
        if os.path.isdir(sub_dir_path):
            for file_name in os.listdir(sub_dir_path):
                if file_name.endswith('.pkl') and 'latent' in file_name and '_tokenid'  in file_name and'_tokenEmbedding' in file_name:
                    file_path = os.path.join(sub_dir_path, file_name)
                    with open(file_path, 'rb') as file:
                        loaded_data = pickle.load(file)
                    # 如果loaded_data里面每一个元素没有下面那么多成分，以每个的shape为准，shape总是对的。比如没有一个3584长度的东西说明embedding没在里面
                    for  eeg, character, latent, tokenid, embed in loaded_data:
                        # eeg.shape (1, 128, 256) 多通道EEG信号
                        # character -> str 一个中文汉字
                        # latent.shape (1, 64) EEG压缩后的向量
                        # tokenid array([24]) 汉字对应的token
                        # embed.shape (3584,)  词嵌入向量
                        characters.append([character,embed])
                       
print(f"预测样本数量：{len(characters)}")
#endregion

预测样本数量：22324


In [5]:
allCharacter=([ch[0] for ch in characters])

In [4]:
import numpy as np
import ollama


class Ollama:

    def getEmbed(text):
        e=ollama.embeddings(model='bge-m3', prompt=text).embedding
        e=np.array(e)
        return e
        

In [62]:
from tqdm import tqdm

# 获取allCharacter的唯一字符并保持顺序
unique_characters = list(dict.fromkeys(allCharacter))

# 找到allCharacter中每个字符在去重后的字符列表中的索引
unique_indices = [allCharacter.index(ch) for ch in unique_characters]

# 使用原始的allCharacterEmbedding根据去重后的索引映射
# allCharacterEmbedding_unique = [allCharacterEmbedding[i] for i in tqdm(unique_indices, desc="Mapping embeddings")]
allCharacterEmbedding = [Ollama.getEmbed(ch) for ch in tqdm(unique_characters, desc="Processing embeddings")]


Processing embeddings:   0%|          | 22/22324 [00:03<41:22,  8.98it/s] 

KeyboardInterrupt: 

In [39]:
for ch in unique_characters:
    print(ch, end=",")


9,我,想,他,是,趁,一,群,野,鸟,迁,徙,的,机,会,出,走,动,身,那,天,早,晨,把,星,球,收,拾,得,井,有,条,仔,细,地,疏,通,了,活,火,山,上,两,座,热,餐,很,方,便,还,死,不,过,正,像,所,说,谁,准,呢,！,以,这,也,照,样,要,就,缓,均,匀,燃,烧,喷,发,跟,烟,囱,冒,当,然,在,们,实,太,小,没,法,去,它,造,成,么,多,麻,烦,由,于,个,缘,故,王,子,拔,掉,刚,长,来,几,株,猴,面,包,树,幼,苗,心,情,点,忧,郁,再,回,些,习,惯,儿,都,显,格,外,亲,切,而,最,后,次,给,花,浇,水,备,盖,罩,时,候,只,觉,哭,见,啦,对,可,她,答,又,遍,咳,嗽,起,但,感,前,傻,终,开,口,请,你,原,谅,愿,能,幸,福,到,吃,惊,居,声,责,举,着,茫,知,措,站,懂,般,恬,淡,柔,爱,错,领,什,紧,和,…,放,边,吧,用,风,并,容,易,夜,晚,新,鲜,空,气,好,处,朵,虫,兽,既,认,识,蝴,蝶,应,该,受,三,毛,挺,看,已,经,远,至,根,本,怕,爪,真,让,四,刺,随,别,磨,蹭,人,决,定,因,为,意,自,己,流,泪,如,此,骄,傲,1,0,颗,附,近,3,2,5,号,6,7,8,行,始,拜,访,找,事,干,增,第,住,国,穿,紫,红,镶,白,鼬,皮,袍,端,坐,张,简,朴,派,庄,严,哈,臣,民,大,叫,纳,闷,从,怎,道,世,界,非,常,单,某,朝,下,里,整,被,袭,华,丽,占,满,累,打,欠,违,宫,廷,礼,仪,禁,止,忍,歉,疚,路,直,睡,命,令,年,玩,吓,涨,脸,呣,嘟,哝,高,兴,其,尊,重,权,威,服,专,制,君,主,善,良,达,理,番,话,畅,极,将,军,变,海,吗,怯,生,问,挪,摆,奇,怪,统,治,陛,允,许,向,您,提,题,赶,抢,明,翼,做,手,势,指,全,归,仅,宇,宙,马,纪,律,涣,散,力,使,叹,果,拥,十,七,二,百,甚,日,落,连,椅,遗,弃,难,壮,胆,求,阳,飞,另,或,者,写,部,悲,剧,拒,执,肯,每,接,首,先,建,立,合,基,础,老,姓,投,反,之,件,忘,记,按,则,等,熟,翻,厚,历,书,概,今,分,乖,憾,腻,任,司,审,判,呀,巡,视,车,慌,噢,瞥,眼,比,智,哪,必,留,耗,听,音,刑,啊,取,悠,赦,免,喜,

In [31]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=50)  # 假设你想分成5个类别
kmeans.fit(allCharacterEmbedding_unique)
labels = kmeans.labels_

In [32]:
categorized_words = {}
for idx, label in enumerate(labels):
    categorized_words.setdefault(label, []).append(unique_characters[idx])
print(categorized_words)


{np.int32(38): ['9', '一', '上', '三', '四', '1', '3', '2', '5', '6', '7', '8', '第', '十', '七', '二', '之', '五', '六', '八', '4', '转', '九', '各', 'B'], np.int32(32): ['我', '他', '的', '那', '了', '很', '！', '这', '也', '就', '们', '它', '亲', '而', '啦', '对', '她', '你', '…', '吧', '好', '该', '哈', '某', '您', '或', '呀', '噢', '啊', '哟', '甭', '‘', '’', '呗', '哩', '哦', '嗨', '哎', '咯', '昨', '呵', '哇', '喔', '奥', '喽', '咦', '嘛', '嗯', '〇'], np.int32(17): ['想', '趁', '出', '收', '得', '通', '还', '过', '要', '发', '跟', '在', '没', '去', '来', '回', '给', '备', '答', '开', '请', '到', '懂', '领', '用', '随', '别', '为', '附', '拜', '找', '被', '问', '做', '求', '接', '按', '等', '留', '取', '待', '致', '帮', '入', '助', '拿', '买', '够', '存', '进', '招', '改', '快', '办', '往', '交', '测', '换', '告', '讨', '送', '信', '劝', '凭', '选', '试', '靠', '遭', '报'], np.int32(35): ['是', '会', '有', '不', '像', '所', '均', '当', '然', '时', '觉', '可', '但', '愿', '能', '应', '因', '如', '权', '难', '先', '必', '确', '虽', '敢', '证', '需', '稳', '况', '似', '倘', '若', '否', '堪', '欲'], np.int32(34): ['群', '属', '形', '种', '栏', '系', 

In [40]:
for i in range(len(categorized_words)):
    print(categorized_words[i])

['拾', '照', '拔', '紧', '磨', '蹭', '打', '赶', '抢', '拥', '拍', '搅', '钻', '抽', '挤', '追', '搞', '挖', '吹', '吸', '掏', '揪', '握', '抱', '搂', '击', '咬', '猜', '捉', '扳', '夺', '捧', '挨', '碰', '惹', '撑', '梳', '捅', '拧', '冲', '锤', '敲', '摇', '挑', '嚼', '揉', '挣']
['谁', '呢', '么', '什', '怎', '吗', '甚', '哪', '何', '偶', '希', '乎']
['盖', '般', '边', '容', '界', '令', '番', '间', '角', '眷', '幅', '伯', '幕', '涉', '泛', '傍', '盘', '旁', '贝', '粹', '丛', '煌', '涯', '际']
['缓', '禁', '止', '拒', '耗', '妨', '离', '灭', '罢', '熄', '关', '削', '消', '抵', '退', '停', '除', '卸', '拆']
['走', '餐', '便', '吃', '居', '行', '访', '住', '穿', '坐', '睡', '玩', '椅', '旅', '学', '忙', '桌', '喝', '闲', '聊', '跑', '逛', '笑', '舞', '俩', '购', '客', '宅', '踏', '凳', '洗', '饮', '搭', '躺', '游', '读']
['野', '鸟', '猴', '虫', '兽', '蝶', '鼬', '龟', '蝇', '蜂', '羊', '狐', '狸', '蟒', '蛇', '驯', '雪', '猎', '鸡', '草', '驼', '象', '螺', '蘑', '楚', '虎', '鸽']
['缘', '包', '罩', '刺', '干', '镶', '皮', '果', '连', '刻', '帽', '袋', '营', '围', '巾', '带', '纸', '屉', '结', '帕', '支', '插', '吊', '绳', '悬', '缝', '戴', '线', '铛', '伴', '丝', '堂', '蔓', '束'

In [53]:
with open('KMeansOfCharacter.pkl', 'wb') as file:
    pickle.dump(categorized_words, file)

In [55]:
language_understanding_and_expression = ["说", "答", "理解", "语句"]
emotion_and_social_interaction = ["快乐", "愤怒", "他人", "我们"]
memory_and_learning = ["记得", "忘记", "回忆"]

actions_and_movement = ["走动", "跳", "跑"]
sensory_experiences = ["触觉", "嗅觉", "视觉"]

spatial_perception = ["位置", "方向"]
social_cognition = ["自我", "我们", "他们"]

reward_and_pleasure = ["奖励", "高兴", "满足"]


language_understanding_and_expressionEmbedding = [Ollama.getEmbed(ch) for ch in language_understanding_and_expression]
emotion_and_social_interactionEmbedding = [Ollama.getEmbed(ch) for ch in emotion_and_social_interaction]
memory_and_learningEmbedding = [Ollama.getEmbed(ch) for ch in memory_and_learning]

actions_and_movementEmbedding = [Ollama.getEmbed(ch) for ch in actions_and_movement]
sensory_experiencesEmbedding = [Ollama.getEmbed(ch) for ch in sensory_experiences]

spatial_perceptionEmbedding = [Ollama.getEmbed(ch) for ch in spatial_perception]
social_cognitionEmbedding = [Ollama.getEmbed(ch) for ch in social_cognition]

reward_and_pleasureEmbedding = [Ollama.getEmbed(ch) for ch in reward_and_pleasure]


In [61]:
import numpy as np

# 对每个类别的 embeddings 列表求平均
language_understanding_and_expression_avg = np.mean(language_understanding_and_expressionEmbedding, axis=0)
emotion_and_social_interaction_avg = np.mean(emotion_and_social_interactionEmbedding, axis=0)
memory_and_learning_avg = np.mean(memory_and_learningEmbedding, axis=0)

actions_and_movement_avg = np.mean(actions_and_movementEmbedding, axis=0)
sensory_experiences_avg = np.mean(sensory_experiencesEmbedding, axis=0)

spatial_perception_avg = np.mean(spatial_perceptionEmbedding, axis=0)
social_cognition_avg = np.mean(social_cognitionEmbedding, axis=0)

reward_and_pleasure_avg = np.mean(reward_and_pleasureEmbedding, axis=0)




In [63]:
from sklearn.cluster import KMeans
import numpy as np

# 将每个类别的平均 embedding 向量汇总为一个列表
category_centroids = np.array([
    language_understanding_and_expression_avg,
    emotion_and_social_interaction_avg,
    memory_and_learning_avg,
    actions_and_movement_avg,
    sensory_experiences_avg,
    spatial_perception_avg,
    social_cognition_avg,
    reward_and_pleasure_avg
])

# 假设 allCharacterEmbedding_unique 是包含所有数据点的 embedding 向量
# 使用 KMeans 进行聚类，设置 n_clusters 为类别数
kmeans = KMeans(n_clusters=8, init=category_centroids, n_init=1)  # 8 是类别数
kmeans.fit(allCharacterEmbedding_unique)

# 获取聚类标签
labels = kmeans.labels_


In [65]:
categorized_words = {}
for idx, label in enumerate(labels):
    categorized_words.setdefault(label, []).append(unique_characters[idx])
for i in range(len(categorized_words)):
    print(categorized_words[i])

['想', '是', '趁', '迁', '会', '出', '把', '收', '得', '有', '通', '还', '不', '过', '像', '说', '准', '照', '要', '发', '跟', '没', '去', '造', '来', '回', '亲', '给', '备', '见', '可', '答', '起', '感', '开', '请', '愿', '能', '到', '吃', '责', '举', '着', '知', '懂', '领', '放', '用', '并', '好', '认', '应', '受', '看', '让', '随', '附', '行', '拜', '访', '找', '增', '住', '单', '占', '满', '玩', '涨', '话', '变', '生', '问', '治', '提', '做', '指', '归', '使', '连', '求', '写', '执', '接', '先', '建', '合', '投', '记', '按', '等', '分', '审', '判', '必', '留', '听', '取', '免', '装', '待', '顶', '致', '拍', '调', '学', '帮', '忙', '微', '入', '喝', '助', '完', '解', '作', '聊', '账', '逛', '无', '拿', '讲', '确', '码', '买', '够', '算', '带', '摘', '存', '进', '结', '招', '呗', '额', '息', '改', '快', '瞧', '办', '往', '续', '关', '交', '险', '测', '查', '复', '供', '证', '换', '插', '抵', '响', '念', '需', '持', '稍', '观', '印', '搞', '序', '俩', '寻', '标', '涉', '男', '递', '养', '谈', '双', '告', '诉', '修', '更', '补', '导', '讨', '仿', '副', '伴', '议', '联', '充', '辨', '注', '购', '售', '瞅', '除', '费', '送', '似', '批', '贴', '卖', '示', '传', '保', '信', '劝', '否',

In [66]:
with open('KMeansOfCharacterWROI.pkl', 'wb') as file:
    pickle.dump(categorized_words, file)

# load data

In [67]:
with open('KMeansOfCharacterWROI.pkl', 'rb') as file:
    categorized_words_ROI = pickle.load(file)
with open('KMeansOfCharacter.pkl', 'rb') as file:
    categorized_words = pickle.load(file)


In [72]:
len(categorized_words)

50

In [73]:
categorized_words[0]

['拾',
 '照',
 '拔',
 '紧',
 '磨',
 '蹭',
 '打',
 '赶',
 '抢',
 '拥',
 '拍',
 '搅',
 '钻',
 '抽',
 '挤',
 '追',
 '搞',
 '挖',
 '吹',
 '吸',
 '掏',
 '揪',
 '握',
 '抱',
 '搂',
 '击',
 '咬',
 '猜',
 '捉',
 '扳',
 '夺',
 '捧',
 '挨',
 '碰',
 '惹',
 '撑',
 '梳',
 '捅',
 '拧',
 '冲',
 '锤',
 '敲',
 '摇',
 '挑',
 '嚼',
 '揉',
 '挣']